In [ ]:
!python3 -V
!python3 -m pip install pandas numpy matplotlib

In [ ]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

# Q1

In [ ]:
# Read the data into a data frame.

data = pd.read_csv('./pitching.csv')
display(data)

In [ ]:
# Display all pitchers in 2015 under each team sorted by their last names.

pitchers_group_team = data[data['yearID'] == 2015].groupby('teamID')
for team in pitchers_group_team.groups:
    print("\033[1;93m" + team + "\033[0m")
    display(pitchers_group_team.get_group(team)[['playerID', 'nameFirst', 'nameLast']].sort_values('nameLast'))

In [ ]:
# Display all pitchers in 2015 sorted by their ERAs. This allows you to identify the best and worst pitchers.

display(data[data['yearID'] == 2015][['playerID', 'nameFirst', 'nameLast', 'ERA']].sort_values('ERA'))

In [ ]:
# Display all pitchers in 2015 under each team sorted by their ERAs. 

pitchers_group_team = data[data['yearID'] == 2015].groupby('teamID')
for team in pitchers_group_team.groups:
    print("\033[1;93m" + team + "\033[0m")
    display(pitchers_group_team.get_group(team)[['playerID', 'nameFirst', 'nameLast', 'ERA']].sort_values('ERA'))

In [ ]:
# Display all pitchers in 2015 under each team sorted by their strike outs (SO).

pitchers_group_team = data[data['yearID'] == 2015].groupby('teamID')
for team in pitchers_group_team.groups:
    print("\033[1;93m" + team + "\033[0m")
    display(pitchers_group_team.get_group(team)[['playerID', 'nameFirst', 'nameLast', 'SO']].sort_values('SO'))

In [ ]:
# Calculate the average ERA for each team, list the teams by the average ERA in a descending ord

display(data.groupby('teamID')[['ERA']].mean().sort_values('ERA', ascending = False))

In [ ]:
# Calculate the total number of home runs allowed (HRs) for each team, and list the team by the count in 
# an ascending order.

display(data.groupby('teamID')[['HR']].sum().sort_values('HR'))

In [ ]:
# Calculate each team’s average ERA from 2011 to 2015, and list the result as a table, where each column 
# is a year, each row is a team.

display(data[(2011 <= data['yearID']) & (data['yearID'] <= 2015)].groupby(['teamID', 'yearID'])[['ERA']].mean().unstack().sort_index())

In [ ]:
# Calculate the average ERAs for American League (AL) and National League (NL) from 2011 to 2015, and 
# list the result as a table, where each column is a league and each row is a year. 

display(data[(2011 <= data['yearID']) & (data['yearID'] <= 2015)].groupby(['lgID', 'yearID'])[['ERA']].mean().unstack())

In [ ]:
# List the pitchers who had the most wins in each of the years from 2011 to 2015.

display(data[(2011 <= data['yearID']) & (data['yearID'] <= 2015)].groupby(['yearID'])[['nameFirst', 'nameLast', 'W']].max())

# Q2

In [ ]:
pitchings_data = pd.read_csv('./pitching.csv')
players_data = pd.read_csv('./players.csv')
teams_data = pd.read_csv('./teams.csv')
display(pitchings_data, players_data, teams_data)

### Task1
In the year of 2011-2015, Is a team’s win-loss record related to its payrolls?

In [ ]:
# Compute the total number of wins for each of the teams in MLB over 2011-2015, sort them in a 
# descending order.

teams_win_data = teams_data[(2011 <= teams_data['yearID']) & (teams_data['yearID'] <= 2015)].groupby('teamID')[['W']].sum().sort_values('W', ascending=False)
display(teams_win_data)

# Counting the win ratio to help plot following picture
teams_win_lose_data = pd.DataFrame(data = teams_data[(2011 <= teams_data['yearID']) & (teams_data['yearID'] <= 2015)].groupby('teamID')[['W', 'L']].sum().apply(lambda x: x['W'] / (x['W'] + x['L']), axis = 'columns'), columns = ['Win-Lose Ratio']).sort_values('Win-Lose Ratio', ascending = False)

In [ ]:
# Compute the average payroll per year for all teams over 2011-2015, sort them in a descending order. 


average_salary_data = players_data[(2011 <= players_data['yearID']) & (players_data['yearID'] <= 2015)].groupby(['teamID'])[['salary']].mean().sort_values('salary', ascending = False)
display(average_salary_data)


In [ ]:
# Create a visualization of your choice which will all allow you to show whether a team’s winning 
# record is related to its payroll. So is a team’s winning record related to its payroll?

print('Correlation Coefficient: ', end = ' ')
print(pd.merge(teams_win_lose_data.sort_index(), average_salary_data.sort_index(), left_on = 'teamID', right_on = 'teamID').corr().at['Win-Lose Ratio', 'salary'])

plt.scatter(teams_win_lose_data.sort_index(), average_salary_data.sort_index())
plt.xlabel('Win-Lose Ratio')
plt.ylabel('Average Salary')
plt.show()

With picture and Correlation Coefficient, we find team’s win-loss record related to its payrolls.

### Task 2
In the year of 2011-2015, Is a player’s batting performance related to his team’s win-loss record? 

In [ ]:
# Compute the Batting Averages for each of the MLB teams over 2011-2015, sort them in a descending
# order. The Batting Average is defined as Hits/At Bats. The average is calculated from all players in 
# each team.

batting_average = pd.DataFrame(data = players_data[(2011 <= players_data['yearID']) & (players_data['yearID'] <= 2015)].groupby('teamID')[['H', 'AB']].sum().apply(lambda x: x['H'] / x['AB'], axis = 'columns'), columns = ['Batting Average']).sort_values('Batting Average', ascending = False)
display(batting_average)

In [ ]:
# Create a visualization of your choice which will allow you to decide whether a team’s batting average 
# is related to its win-loss record. So is a team’s winning record related to its batting average?

print('Correlation Coefficient: ', end = ' ')
print(pd.merge(teams_win_lose_data.sort_index(), batting_average.sort_index(), left_on = 'teamID', right_on = 'teamID').corr().at['Win-Lose Ratio', 'Batting Average'])

plt.scatter(teams_win_lose_data.sort_index(), batting_average.sort_index())
plt.xlabel('Win-Lose Ratio')
plt.ylabel('Batting Average')
plt.show()

With picture and Correlation Coefficient, we find player’s batting performance related to his team’s win-loss record.

### Task 3
In the year of 2011-2015, is a team’s win-loss record related to its pitching performance? 

In [ ]:
# Display the average ERA (Earned Run Average) for each of the MLB teams in 2011-2015, sort them 
# in a descending order. A lower ERA indicates a better pitching performance.

teams_era = teams_data[(2011 <= teams_data['yearID']) & (teams_data['yearID'] <= 2015)].groupby('teamID')[['ERA']].sum().sort_values('ERA', ascending = False)
display(teams_era)

In [ ]:
# Create a visualization of your choice which will allow you to decide if a team’s win-loss record is related
# to its pitching performance.  So is a team’s winning record related to its pitching performance? 

print('Correlation Coefficient: ', end = ' ')
print(pd.merge(teams_win_lose_data.sort_index(), teams_era.sort_index(), left_on = 'teamID', right_on = 'teamID').corr().at['Win-Lose Ratio', 'ERA'])

plt.scatter(teams_win_lose_data.sort_index(), teams_era.sort_index())
plt.xlabel('Win-Lose Ratio')
plt.ylabel('REA')
plt.show()

player’s batting performance not related to his team’s win-loss record.